In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size=64
epochs=100
latent_dim=256
num_samples=10000
data_path='/fra.txt'

In [3]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
  input_text,  target_text, _ = line.split('\t')
  target_text='\t'+target_text+'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)


In [4]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [5]:
print('Number of samples:',len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [6]:
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)]
)

In [7]:
encoder_input_data=np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32'
)
decoder_input_data=np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32'
)
decoder_target_data=np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32'
)


In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] =1.
  encoder_input_data[i, t+1:, input_token_index[' ']] =1.
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] =1.
    if t > 0:
      decoder_target_data[i, t-1, target_token_index[char]] =1.
  decoder_input_data[i, t+1:, target_token_index[' ']] =1.
  decoder_target_data[i, t:, target_token_index[' ']] =1.

In [9]:
encoder_input_data[0].shape

(15, 71)

In [10]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [11]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
          batch_size=batch_size, epochs= epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 47s 359ms/step - loss: 1.4742 - accuracy: 0.7328 - val_loss: 1.1510 - val_accuracy: 0.7107
Epoch 2/100
125/125 [==============================] - 44s 355ms/step - loss: 0.8615 - accuracy: 0.7659 - val_loss: 0.8162 - val_accuracy: 0.7728
Epoch 3/100
125/125 [==============================] - 45s 356ms/step - loss: 0.6813 - accuracy: 0.8099 - val_loss: 0.6823 - val_accuracy: 0.8027
Epoch 4/100
125/125 [==============================] - 44s 354ms/step - loss: 0.5832 - accuracy: 0.8303 - val_loss: 0.6321 - val_accuracy: 0.8181
Epoch 5/100
125/125 [==============================] - 44s 352ms/step - loss: 0.5309 - accuracy: 0.8452 - val_loss: 0.6037 - val_accuracy: 0.8233
Epoch 6/100
125/125 [==============================] - 44s 354ms/step - loss: 0.4973 - accuracy: 0.8537 - val_loss: 0.5676 - val_accuracy: 0.8324
Epoch 7/100
125/125 [==============================] - 44s 354ms/step - loss: 0.4698 - accuracy: 0.8615 - val_loss: 0.5432 -

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_state_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_state_inputs,
    [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items()
)
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items()
)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence: ', input_texts[seq_index])
    print('Decoded sentence: ', decoded_sentence)

-
Input sentence:  Go.
Decoded sentence:  Va !

-
Input sentence:  Go.
Decoded sentence:  Va !

-
Input sentence:  Go.
Decoded sentence:  Va !

-
Input sentence:  Hi.
Decoded sentence:  Salut.

-
Input sentence:  Hi.
Decoded sentence:  Salut.

-
Input sentence:  Run!
Decoded sentence:  Cours !

-
Input sentence:  Run!
Decoded sentence:  Cours !

-
Input sentence:  Run!
Decoded sentence:  Cours !

-
Input sentence:  Run!
Decoded sentence:  Cours !

-
Input sentence:  Run!
Decoded sentence:  Cours !

-
Input sentence:  Run!
Decoded sentence:  Cours !

-
Input sentence:  Run!
Decoded sentence:  Cours !

-
Input sentence:  Run!
Decoded sentence:  Cours !

-
Input sentence:  Run.
Decoded sentence:  Cours !

-
Input sentence:  Run.
Decoded sentence:  Cours !

-
Input sentence:  Run.
Decoded sentence:  Cours !

-
Input sentence:  Run.
Decoded sentence:  Cours !

-
Input sentence:  Run.
Decoded sentence:  Cours !

-
Input sentence:  Run.
Decoded sentence:  Cours !

-
Input sentence:  Run.
Deco